<a href="https://colab.research.google.com/github/shaikh-rumiza-shakeel/github-demo/blob/master/CreditCardFraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***The complete end-to-end model training process to get the best model that can classify the transactions into normal and abnormal types.*** 

# About the data

The data for this article can be found here. This dataset contains the real bank transactions made by European cardholders in the year 2013. As a security concern, the actual variables are not being shared but — they have been transformed versions of PCA. As a result, we can find 29 feature columns and 1 final class column.

# Importing Necessary Libraries

It is a good practice to import all the necessary libraries in one place — so that we can modify them quickly.
For this credit card data, the features that we have in the dataset are the transformed version of PCA, so we will not need to perform the feature selection again. Otherwise, it is recommended to use RFE, RFECV, SelectKBest and VIF score to find the best features for your model.

In [179]:
#Packages related to general operating system & warnings
import os 
import warnings
warnings.filterwarnings('ignore')
#Packages related to data importing, manipulation, exploratory data #analysis, data understanding
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from numpy import inf
from termcolor import colored as cl # text customization
#Packages related to data visualizaiton
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#Setting plot sizes and type of plot
plt.rc("font", size=14)
plt.rcParams['axes.grid'] = True
plt.figure(figsize=(6,3))
plt.gray()
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import  PolynomialFeatures, KBinsDiscretizer, FunctionTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, OrdinalEncoder
import statsmodels.formula.api as smf
import statsmodels.tsa as tsa
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import BaggingClassifier, BaggingRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor, AdaBoostClassifier, AdaBoostRegressor 
from sklearn.svm import LinearSVC, LinearSVR, SVC, SVR
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

<Figure size 432x216 with 0 Axes>

#Importing the Dataset

Importing the dataset is pretty much simple. You can use pandas module in python to import it.

Run the below command to import your data.

In [180]:
data=pd.read_csv("/content/sample_data/creditcard.csv")

Replacing the NAN values in the Dataset

In [181]:
data = data.fillna(194194)

Checking the transaction distribution

In [182]:
Total_transactions = len(data)
normal = len(data[data.Class == 0])
fraudulent = len(data[data.Class == 1])
fraud_percentage = round(fraudulent/normal*100, 2)
print(cl('Total number of Trnsactions are {}'.format(Total_transactions), attrs = ['bold']))
print(cl('Number of Normal Transactions are {}'.format(normal), attrs = ['bold']))
print(cl('Number of fraudulent Transactions are {}'.format(fraudulent), attrs = ['bold']))
print(cl('Percentage of fraud Transactions is {}'.format(fraud_percentage), attrs = ['bold']))


Total number of Trnsactions are 284807
Number of Normal Transactions are 284315
Number of fraudulent Transactions are 492
Percentage of fraud Transactions is 0.17


We can also check for null values using the following lines of code.

In [183]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [184]:
data.isnull().any()

Time      False
V1        False
V2        False
V3        False
V4        False
V5        False
V6        False
V7        False
V8        False
V9        False
V10       False
V11       False
V12       False
V13       False
V14       False
V15       False
V16       False
V17       False
V18       False
V19       False
V20       False
V21       False
V22       False
V23       False
V24       False
V25       False
V26       False
V27       False
V28       False
Amount    False
Class     False
dtype: bool

Checking the minimum and maximum is in the amount tells us that the difference is huge that can deviate our result.

In [185]:
min(data.Amount), max(data.Amount)

(0.0, 25691.16)

In this case, it is a good practice to scale this variable. We can use a standard scaler to make it fix.

In [186]:
sc = StandardScaler()
amount = data['Amount'].values
data['Amount'] = sc.fit_transform(amount.reshape(-1, 1))

We have one more variable which is the time which can be an external deciding factor — but in our modelling process, we can drop it.

In [187]:
data.drop(['Time'], axis=1, inplace=True)

Let’s remove the duplicates in our data and observe the changes.

In [188]:
data.shape

(284807, 30)

Run the below line of code to remove any duplicates.

In [189]:
data.drop_duplicates(inplace=True)

Let’s now check the count again.

In [190]:
data.shape

(275663, 30)

**So, we were having around ~9000 duplicate transactions.**

# Train & Test Split

Before splitting train & test — we need to define dependent and independent variables. The dependent variable is also known as X and the independent variable is known as y.

In [191]:
X = data.drop('Class', axis = 1).values
y = data['Class'].values

Now, let split our train and test data.

In [192]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

Train data we will be used for training our model and the data which is unseen will be used for testing.

Some lines of code to clean our dataset further and deal with any undealt NAN values

In [193]:
def clean_dataset(data):
    assert isinstance(data, pd.DataFrame), "data needs to be a pd.DataFrame"
    data.dropna(inplace=True)
    indices_to_keep = ~data.isin([np.nan, np.inf, -np.inf]).any(1)
    return data[indices_to_keep].astype(np.float64)

In [194]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna(999, inplace=True)

In [195]:
data[data == inf] = np.finfo(np.float32).max

# Model Building

### ***Decision Tree***

In [196]:
DT = DecisionTreeClassifier(max_depth = 4, criterion = 'entropy')
DT.fit(X_train, y_train)
dt_yhat = DT.predict(X_test)

Let’s check the accuracy of our decision tree model.

In [197]:
print('Accuracy score of the Decision Tree model is {}'.format(accuracy_score(y_test, dt_yhat)))

Accuracy score of the Decision Tree model is 0.9991729061466132


Checking F1-Score for the decision tree model.

In [198]:
print('F1 score of the Decision Tree model is {}'.format(f1_score(y_test, dt_yhat)))

F1 score of the Decision Tree model is 0.7574468085106382


Checking the confusion matrix:

In [199]:
confusion_matrix(y_test, dt_yhat, labels = [0, 1])

array([[68770,    18],
       [   39,    89]])

*Let’s now try different models and check their performance...*

### ***K-Nearest Neighbors***

In [ ]:
n = 7
KNN = KNeighborsClassifier(n_neighbors = n)
KNN.fit(X_train, y_train)
knn_yhat = KNN.predict(X_test)

Let’s check the accuracy of our K-Nearest Neighbors model.

In [ ]:
print('Accuracy score of the K-Nearest Neighbors model is {}'.format(accuracy_score(y_test, knn_yhat)))

Checking F1-Score for the K-Nearest Neighbors model.

In [154]:
print('F1 score of the K-Nearest Neighbors model is {}'.format(f1_score(y_test, knn_yhat)))

F1 score of the K-Nearest Neighbors model is 0.7777777777777778


### ***Logistic Regression***

In [155]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_yhat = lr.predict(X_test)

Let’s check the accuracy of our Logistic Regression model.

In [156]:
print('Accuracy score of the Logistic Regression model is {}'.format(accuracy_score(y_test, lr_yhat)))

Accuracy score of the Logistic Regression model is 0.9978228486306864


Checking F1-Score for the Logistic Regression model.

In [157]:
print('F1 score of the Logistic Regression model is {}'.format(f1_score(y_test, lr_yhat)))

F1 score of the Logistic Regression model is 0.6274509803921569


### ***Support Vector Machines***

In [158]:
svm = SVC()
svm.fit(X_train, y_train)
svm_yhat = svm.predict(X_test)

Let’s check the accuracy of our Support Vector Machines model.

In [159]:
print('Accuracy score of the Support Vector Machines model is {}'.format(accuracy_score(y_test, svm_yhat)))

Accuracy score of the Support Vector Machines model is 0.9963332187464191


Checking F1-Score for the Support Vector Machines model.

In [160]:
print('F1 score of the Support Vector Machines model is {}'.format(f1_score(y_test, svm_yhat)))

F1 score of the Support Vector Machines model is 0.0


### ***Random Forest***

In [161]:
rf = RandomForestClassifier(max_depth = 4)
rf.fit(X_train, y_train)
rf_yhat = rf.predict(X_test)

Let’s check the accuracy of our Random Forest model.

In [162]:
print('Accuracy score of the Random Forest model is {}'.format(accuracy_score(y_test, rf_yhat)))

Accuracy score of the Random Forest model is 0.9987395439440816


Checking F1-Score for the Random Forest model.

In [163]:
print('F1 score of the Random Forest model is {}'.format(f1_score(y_test, rf_yhat)))

F1 score of the Random Forest model is 0.8


### ***XGBoost***

In [164]:
xgb = XGBClassifier(max_depth = 4)
xgb.fit(X_train, y_train)
xgb_yhat = xgb.predict(X_test)

Let’s check the accuracy of our XGBoost model.

In [165]:
print('Accuracy score of the XGBoost model is {}'.format(accuracy_score(y_test, xgb_yhat)))

Accuracy score of the XGBoost model is 0.9985103701157327


***We just received 99.95% accuracy in our credit card fraud detection!!!📯🚀🚀***

Checking F1-Score for the XGBoost model.

In [166]:
print('F1 score of the XGBoost model is {}'.format(f1_score(y_test, xgb_yhat)))

F1 score of the XGBoost model is 0.7636363636363634
